### 설치 – install



In [3]:
!pip install selenium

In [ ]:
pip install ipywidgets

     -------------------------------------- 123.4/123.4 kB 7.1 MB/s eta 0:00:00
     -------------------------------------- 245.3/245.3 kB 7.6 MB/s eta 0:00:00
     ---------------------------------------- 1.6/1.6 MB 14.5 MB/s eta 0:00:00
  Using cached notebook-6.4.12-py3-none-any.whl (9.9 MB)
  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
     ------------------------------------- 561.6/561.6 kB 11.7 MB/s eta 0:00:00
  Using cached Send2Trash-1.8.0-py3-none-any.whl (18 kB)
     ---------------------------------------- 73.3/73.3 kB 3.9 MB/s eta 0:00:00
     ---------------------------------------- 59.5/59.5 kB 3.1 MB/s eta 0:00:00
     ------------------------------------- 160.9/160.9 kB 10.0 MB/s eta 0:00:00
  Using cached packaging-21.3-py3-none-any.whl (40 kB)
Note: you may need to restart the kernel to use updated packages.Collecting defusedxml

     ---------------------------------------- 71.8/71.8 kB ? eta 0:00:00
     ---------------------------------------- 81.1/81.1 k

### 드라이버 – driver

In [4]:
!pip install chromedriver_autoinstaller

### 라이브러리 import

In [5]:
import pandas as pd
import numpy as np
import os
import math 

import requests
from bs4 import BeautifulSoup
import chromedriver_autoinstaller

# Selenium
from selenium import webdriver
from selenium.webdriver.common.alert import Alert
from selenium.webdriver import ActionChains as AC # ActionChains are a way to automate low level interactions such as mouse movements, mouse button actions, key press, and context menu interactions.
from selenium.webdriver.common.by import By       # Set of supported locator strategies.
from selenium.webdriver.common.keys import Keys   # Set of special keys codes
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait    # Constructor, takes a WebDriver instance and timeout in seconds.

# tqdm : for문 진행상황 체크
from tqdm import tqdm, tqdm_notebook
from tqdm.notebook import tqdm

# 정규표현식(regular expression) : 문자(알파벳,한글), 숫자, 특수기호 정제 및 추출
import re
from time import sleep
import time

# 워닝 무시
import warnings
warnings.filterwarnings('ignore')


### 브라우저 열기 (chrome)

In [102]:
# 크롬창에서 멜론 발라드 부분 띄우기
chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path)

# 크롬 옵션
options = webdriver.ChromeOptions()  
# 크롬 윈도우 사이즈 조절
options.add_argument("--window-size=800, 1200") # window-size -> 기본 : 1920,1080

driver.get("https://www.melon.com/genre/song_list.htm?gnrCode=GN0100#params%5BgnrCode%5D=GN0100&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=NEW&params%5BsteadyYn%5D=N&po=pageObj&startIndex=1")
time.sleep(3)

### 🧙 전체 코드 (가사 없는 곡 PASS 가능)

In [ ]:
songId = []
artist_names = []
songs = []
lyrics = []


### 페이지 이동 (1~1095 pages)
for j in tqdm(range(2,3)) :  # PAGE 범위 조정할 것 (~1095 페이지까지 있음)
    print(f'----------{j+1}page 크롤링 중----------')
    #차단 막기
    seed = np.random.randint(100)
    np.random.seed(seed)
    a = np.random.randint(5)
    time.sleep(a)
    
    # url 설정 (해당 페이지에 대한 url)
    page_url = f'https://www.melon.com/genre/song_list.htm?gnrCode=GN0100#params%5BgnrCode%5D=GN0100&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=NEW&params%5BsteadyYn%5D=N&po=pageObj&startIndex={1+(50*j)}'
    
    # url 접속
    driver.get(page_url)
    time.sleep(0.1)


    ### URL 이용하여 해당 페이지의 곡 번호 추출하기 (50 songs/page)
    # 가사 아이콘 확인하고 곡번호 url 확인하기
    elements = driver.find_elements(By.CSS_SELECTOR, "a.btn.button_icons.type03.song_info")
    
    product_id = []
    for ele in elements:
        thing = ele.get_attribute('href')
        p = re.compile('[0-9]{8,8}')
        song_id = p.findall(thing)[0]
        # print(song_id)
        product_id.append(song_id)

    print(product_id)
    songId.append(product_id)

    ### 곡 번호(songId)로 돌리기 in page
    for i in tqdm_notebook(product_id) :
        print(f'----------{product_id.index(i)}번째 노래를 크롤링 중입니다.----------')
        #차단 막기
        seed = np.random.randint(100)
        np.random.seed(seed)
        a = np.random.randint(5)
        time.sleep(a)

        # url 설정 (해당 곡에 대한 url)
        url = f'https://www.melon.com/song/detail.htm?songId={i}'

        # url 접속
        driver.get(url)
        time.sleep(0.1)    

        # 가수명 크롤링
        artist_name = driver.find_element(By.CSS_SELECTOR, "a.artist_name").text
        print(artist_name)
        artist_names.append(artist_name)
        
        # 곡명 크롤링
        song_name = driver.find_element(By.CSS_SELECTOR, "div.song_name").text
        print(song_name)
        songs.append(song_name)

            
        # 가사 있을 때
        try:
            if driver.find_element(By.CSS_SELECTOR, "button.button_more.arrow_d").text == '펼치기':
                # 펼치기 아이콘 클릭
                driver.find_element(By.CSS_SELECTOR, "button.button_more.arrow_d").click()
                time.sleep(0.1)

                # 가사 크롤링
                lyric = driver.find_element(By.ID, "d_video_summary").text
                print(lyric)
                lyrics.append(lyric)
            
        # 가사 없을 때
        except:
            non_lyric = driver.find_element(By.ID,"lyricArea").text
            if non_lyric.startswith("[가사 준비중]"):
                lyric = ""
                lyrics.append(lyric)
        
            
# 가사 없는 곡 예시
# https://www.melon.com/song/detail.htm?songId=35413703  밤의 해변에서 혼자 (연주곡) / 초야
# https://www.melon.com/song/detail.htm?songId=35319615 꿈속으로 / Hope


print('크롤링 완료!!!')

  0%|          | 0/2 [00:00<?, ?it/s]

----------3page 크롤링 중----------
['35419336', '35417924', '35428994', '35429157', '35427845', '35427133', '35426098', '35425788', '35421841', '35421638', '35419751', '35419624', '35417575', '35417573', '35422272', '35419828', '35418074', '35415696', '35418250', '35418098', '35419786', '35419785', '35419772', '35419184', '35418982', '35418258', '35418257', '35416388', '35413703', '35413699', '35418345', '35413068', '35418748', '35418920', '35417890', '35418092', '35418094', '35419015', '35417795', '35418970', '35418255', '35417783', '35413934', '35408986', '35417892', '35417894', '35418131', '35418974', '35417633', '35414734']


  0%|          | 0/50 [00:00<?, ?it/s]

----------0번째 노래를 크롤링 중입니다.----------
김종호
그대와 나
행복했던 그대와 나
기억 속 우리의 사랑의 이야기
그대와 나 언제나 영원히
그 순간을 함께해요
꿈결같던 그대와 나
기억 속 우리의 못 다한 이야기
그대와 나 언제나 영원히
그 시간을 함께해요
언제나 그대와 둘이서
함께 할거라고 난 믿었죠
저 하늘 아래 펼쳐진 피어 난
아름다운 우리의 사랑이
함께 한 지난 날 우리가
서로 다른 곳을 보고 있죠
저 하늘 위에 숨겨진 별들이
구름들을 탓하며 빛춰요 워어어어
행복했던 그대와 나
기억 속 우리의 사랑의 이야기
그대와 나 언제나 영원히
그 순간을 함께해요
꿈결같던 그대와 나
기억 속 우리의 못 다한 이야기
그대와 나 언제나 영원히
그 시간을 함께해요
그대를 사랑한 나니까
후회없는 사랑을 했어요
그 시간 속에 멈춰진
우리의 애태웠던 사랑을
기억해 워어어어
소중했던 그대와 나
추억 속 우리의 그리운 이야기
그대와 나 언제나 영원히
그 추억을 함께해요
그 추억을 함께해요
----------1번째 노래를 크롤링 중입니다.----------
김민정
회색 하늘 속 빛을 찾아
혼자서 떠나고 싶지만
나에겐 시간이 없어
어렵게 버티고 있지만
이제 모든 것을 내려놓고 싶어
저 멀리 보이는 그 곳을
우리는 갈수가 없어
바쁘게 지나간 시간을
다시 돌릴 수가 없다는걸 알고
회색 빛 하늘 아래서 느껴지는
무겁고 힘든 감정들 속에
힘들게 하루가 지나가고
똑같은 일상의 반복에
조금씩 모르게 나는 망가져가기에
이제는 모든 것에 벗어나고 싶기에
뿌옇게 변해간 하늘에
가려진 태양을 찾고 싶어
멀리서 희미하게 보여지는
틈 사이 반짝이는 빛
새로운 하루는 시작하고
달라진 일상의 기대에
조금씩 모르게 나는 변할 수 있기에
이제는 모든 것을 떨쳐내고 싶기에
변하는 회색하늘 빛
이제 빛을 찾아 떠날 수 있기에
조금씩 모르게 나는 걸을 수 있기에
이제는 모든 것을 이겨낼수 있기에
----------2번째 노래를 크롤링 중입니다.----------
로시 (Rothy)
Step By 

  0%|          | 0/50 [00:00<?, ?it/s]

----------0번째 노래를 크롤링 중입니다.----------
지연
또르르 (2022)
또르르 눈물이 흘러간다
또르르 또르르르
소리없는 아픔을 이기지도 못한 채
파르르 손끝이 떨려온다
파르르 파르르르
따스했던 시간을 기억하나봐
사랑을 하면 더 예뻐진대
사랑을 하면 좀 달라진대
어떻게 해야 내 안의 사랑이
더 예뻐질까요
생각을 하면 눈물이 나고
눈물이 나면 또 생각나는
그런 사람이 곁에 있다는 게
다행이죠
스르르 두 눈이 감겨온다
스르르 스르르르
고운 미소 향기에
꿈을 꾸고 싶었나봐
또르르 사랑이 흘러간다
또르르 또르르르
맘이 시려울만큼 좋아하나봐
사랑을 하면 더 예뻐진대
사랑을 하면 좀 달라진대
어떻게 해야 내 안의 사랑이
더 예뻐질까요
생각을 하면 눈물이 나고
눈물이 나면 또 생각나는
그런 사람이 곁에 있다는 게
다행이죠
가슴에 차가운 네가 내리면
못본 듯이 그저 웃어야 해요
행여 네가 돌아볼까봐
항상 그 자리를 맴도는
어리석어도 행복한 사랑이 좋아서
사랑을 하면 더 예뻐진대
사랑을 하면 좀 달라진대
어떻게 해야 내 안의 사랑이
더 예뻐질까요
생각을 하면 눈물이 나고
눈물이 나면 또 생각나는
그런 사람이 곁에 있다는 게
다행이죠
----------1번째 노래를 크롤링 중입니다.----------
이민혁
고백해줘요
아무 일 없는 하루가
아무렇지도 않게 끝나요
한 페이지에 담긴 일기처럼
평범한 날이죠
소설 속 주인공처럼
영화 속 마지막 scene처럼
운명을 믿는 건 내게 너무
큰 욕심인 걸까요
노란 별빛 조명 아래서
나에게 고백해줄래요
모두가 축하해주는 날
내 마음이 누구보다 빛나게
비밀스러운 그 마음이
나를 위한 맘이길 바라요
다정하게 바라본 눈빛은
별 의미 없나요
마음껏 웃게 될지도
마음껏 울지도 몰라요
어떤 엔딩이 될지는
그대만이 알고 있어요
노란 별빛 조명 아래서
나에게 고백해줄래요
그대와 내 맘 같다면
불행들도 의미가 있어요
내 표정들을 좋아한다고
영원히 날 좋아한다고
계속해서 말해줄래요
말해줘도 믿을 수 없어요
----------2번째 노래를 크

KeyboardInterrupt: 

### 딕셔너리 & 데이터프레임 생성

In [ ]:
page_coll = {}

page_coll['artist_name'] = artist_names
page_coll['song_name'] = songs
page_coll['lyric'] = lyrics

page_coll

{'artist_name': ['김종호',
  '김민정',
  '로시 (Rothy)',
  '레이나 (Raina)',
  '유주 (YUJU)',
  '황인혁',
  '이로이 (TEAM152)',
  '포근',
  '준상',
  '허지영',
  '류보영',
  'yunseo',
  '진정(Jin.Jung)',
  '박찬민',
  '마인드유',
  'OuiOui (위위)',
  '민우혁',
  '이동수',
  '영동시장 (詠同市場)',
  '카페모카',
  'Daandi',
  '혜크',
  '조용현',
  '이주헌',
  '김한성',
  '이태백',
  '홍빠',
  '푸름',
  '초야',
  '김현지',
  '경서예지',
  '거미',
  '윤언니',
  '기탁',
  '전상근',
  '송민경',
  '우이경 (Woo Yi Kyung)',
  '이민형',
  'POLD',
  '임재범',
  '가호 (Gaho)',
  '바다 (BADA)',
  '이준화',
  '최진솔',
  '송푸름',
  '더 브릿지 (The Bridge)',
  '정피트',
  '석홍',
  '유현국',
  '길구봉구',
  '지연',
  '이민혁',
  '현신영'],
 'song_name': ['그대와 나',
  '회색 하늘 속 빛을 찾아',
  'Step By Step',
  '끝나지 않는 밤',
  'Tell me this is real',
  '스케치',
  'Summer Dream (Feat. 노소영)',
  '모두의 밤에게 (With. 윈리 (Winry))',
  '나 당신의 노랠 만들고 있는데',
  '강박',
  '사랑이 있어요',
  '내 자신에게...',
  '눈내리는날 커피를 마시며 (Feat. 송윤선)',
  '향기',
  '여름의 중심에서',
  "빈칸 (웹툰 '연애의 발견' X OuiOui (위위))",
  '바라본다',
  '목단 (My Only Ace)',
  '너와 나의 푸르름',
  '이렇게 사랑을 배워 (feat. 진새벽)',
  '내 소식',
  'O

In [ ]:
target_df = pd.DataFrame.from_dict(page_coll, orient='index')
df = target_df.transpose()
df.head(30)

,artist_name,song_name,lyric
0,김종호,그대와 나,행복했던 그대와 나\n기억 속 우리의 사랑의 이야기\n그대와 나 언제나 영원히\n그...
1,김민정,회색 하늘 속 빛을 찾아,혼자서 떠나고 싶지만\n나에겐 시간이 없어\n어렵게 버티고 있지만\n이제 모든 것을...
2,로시 (Rothy),Step By Step,Step by step we’re getting closer\nThe system ...
3,레이나 (Raina),끝나지 않는 밤,나지막이 들려온다\n익숙했던 너의 소리\n하루의 모든 날 달래주었었던\n너의 따스한...
4,유주 (YUJU),Tell me this is real,So tell me this is real\n내게 말을 해줘\n진실을 보여줘 제발\...
5,황인혁,스케치,사실 그대에게 하지 못했던\n그 말 들어 줄래요\n우린 같은 마음인가요\n그댄 기다...
6,이로이 (TEAM152),Summer Dream (Feat. 노소영),오늘은 예감이 좋아\n바람도 적당히 부는\n모든게 완벽한 하루\n한참을 옷을 고르고...
7,포근,모두의 밤에게 (With. 윈리 (Winry)),어떻게 잠드는지 잊은 채\n오늘을 보내지 못하고\n혼자 머물러 있어\n멍하니 밤하늘...
8,준상,나 당신의 노랠 만들고 있는데,나 당신의 노랠 만들고 있는데\n왠지 모르게 잘 안 돼요\n나 당신의 얘길 여기 써...
9,허지영,강박,강박이 생겨버린 건 아닌지\n걱정이 되는 요즘\n이상한데 꽂혀서\n갇혀버리는 건 아...
